In [1]:
import tensorflow as tf
import numpy as np

import tf_encrypted as tfe

In [2]:
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.models import Sequential

## Plaintext model

In [3]:
model_k = Sequential([Dense(32, activation='relu', batch_input_shape=[1,32]), 
                      Dense(20, activation='relu')])

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
data = np.ones(shape=[1, 32]).astype('float')
model_k.predict(data)

array([[0.        , 0.        , 0.06978083, 0.        , 0.        ,
        0.53216815, 0.        , 0.        , 0.        , 0.        ,
        0.90373284, 1.1124173 , 0.        , 0.13519186, 0.23586318,
        0.01619154, 0.        , 0.01004681, 0.0254201 , 0.        ]],
      dtype=float32)

## Create tfe keras model based on keras config

In [5]:
from tf_encrypted.keras.models import Sequential

In [6]:
config = model_k.get_config()

with tfe.protocol.SecureNN():
    
    tfe_model = Sequential([])
    tfe_model.from_config(config)
  
    x = tfe.define_private_variable(data)
  
    y = tfe_model(x)

In [7]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    actual = sess.run(y.reveal())

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


In [8]:
print(actual)

[[0.10623381 0.         0.05776558 1.56089011 0.         0.77518671
  1.05288828 0.47843317 0.30102119 0.         0.         0.
  1.18655693 0.         0.         1.60829142 0.         0.
  0.79652492 0.        ]]


In [9]:
tfe_model.weights

[PondPrivateVariable(shape=(32, 32)),
 PondPrivateVariable(shape=(32,)),
 PondPrivateVariable(shape=(32, 20)),
 PondPrivateVariable(shape=(20,))]

## Create tfe keras model based on keras config and set weights

In [10]:
keras_weights = model_k.get_weights()
config = model_k.get_config()

In [11]:
tf.reset_default_graph()

with tfe.protocol.SecureNN():
    tfe_model = Sequential([])
    tfe_model.from_config(config)
    x = tfe.define_private_variable(data)
    
    y = tfe_model(x)
    
    tfe_model.set_weights(keras_weights)

PondPrivateVariable(shape=(32, 32))
PondPrivateVariable(shape=(32,))
PondPrivateVariable(shape=(32, 20))
PondPrivateVariable(shape=(20,))


In [12]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tfe_model.set_weights(keras_weights))
    actual = sess.run(y.reveal())

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


PondPrivateVariable(shape=(32, 32))
PondPrivateVariable(shape=(32,))
PondPrivateVariable(shape=(32, 20))
PondPrivateVariable(shape=(20,))


AttributeError: 'NoneType' object has no attribute 'to_native'

In [ ]:
print(actual)